In [7]:
from rfgap import RFGAP
from dataset import dataprep

import pandas as pd
import numpy as np

In [8]:
# Read in the data
data   = pd.read_csv('data/iris.csv', sep = ',')
x, y   = dataprep(data)
n, d   = x.shape
n_classes = len(y.unique())

In [9]:
rfgap = RFGAP(y = y, non_zero_diagonal= False)

In [10]:
rfgap.fit(x, y)

In [11]:
prox = rfgap.get_proximities()

In [12]:
prox.todense()

matrix([[0.        , 0.01760353, 0.01912214, ..., 0.        , 0.        ,
         0.        ],
        [0.01760353, 0.        , 0.01873496, ..., 0.        , 0.        ,
         0.        ],
        [0.01912214, 0.01873496, 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.0238422 ,
         0.01889734],
        [0.        , 0.        , 0.        , ..., 0.0238422 , 0.        ,
         0.01629636],
        [0.        , 0.        , 0.        , ..., 0.01889734, 0.01629636,
         0.        ]])